In [6]:
import os
import argparse
import os
import torch
from pathlib import Path
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
import numpy as np
import random
import os
import re
sample_dir = './all_visualize'
os.makedirs(sample_dir, exist_ok=True)

In [13]:
def person_cam_split(dataset_path, cam_num):
    image_paths = sorted(list(dataset_path.glob('bounding_box_train/*.jpg')))
    person = {} # {id : {0: [cam0_list], 1: [cam1_list], ... }
    for image_path in image_paths:
        image_name = image_path.name
        person_id = image_name.split('_')[0]
        cam_id = int(image_name.split('_')[1][1]) - 1
        if person_id in person.keys():
            if cam_id in person[person_id].keys():
                person[person_id][cam_id].append(image_path)
            else:
                person[person_id][cam_id] = [image_path]
        else:
            person[person_id] = {}
            person[person_id][cam_id] = [image_path]
    return person

def cam_filter(person_dic, cam_num):
    full_cam = {}
    for person_id, all_cam in person_dic.items():
        if len(all_cam) == cam_num:
            full_cam[person_id] = all_cam
    return full_cam

def cam_split(dataset_path, cam_num):
    image_paths = sorted(list(dataset_path.glob('bounding_box_train/*.jpg')))
    cam_list = []
    for i in range(cam_num):
        cam_list.append([])
    for image_path in image_paths:
        image_name = image_path.name
        cam_id = int(image_name.split('_')[1][1]) - 1
        cam_list[cam_id].append(image_path)
    return cam_list

def cam_split_dataset(person_dict, save_dir):
    save_dir = Path(save_dir)
    for person, cams in person_dict.items():
        person_dir = save_dir/person
        person_dir.mkdir(parents=True, exist_ok=True)
        for cam, image_paths in cams.items():
            cam_dir = person_dir/str(cam)
            cam_dir.mkdir(parents=True, exist_ok=True)
            for image_path in image_paths:
                img = Image.open(image_path)
                # Resize the image to 256x128
                img = img.resize((128, 256)) 
                image_path = cam_dir/image_path.name
                img.save(image_path)

def strip_name(filename):
    # Use regex to remove the 'transfered[some_number]' part
    stripped_filename = re.sub(r'transfered\d+', '', filename)
    return stripped_filename

def transfered_split(dataset_path):
    image_paths = sorted(list(dataset_path.glob('bounding_box_train/*.jpg')))
    transfered_dict = {}
    for image_path in image_paths:
        image_name = strip_name(image_path.name)
        if image_name in transfered_dict.keys():
            transfered_dict[image_name].append(image_path)
        else:
            transfered_dict[image_name] = [image_path]
    return transfered_dict

In [14]:
market_all_dataset_path = Path('/home/jun/ReID_Dataset/Market-1501-v15.09.15-stargan-allcam')
market_all_tranfered_split = transfered_split(market_all_dataset_path)
duke_all_dataset_path = Path('/home/jun/ReID_Dataset/DukeMTMC-reID-stargan-allcam')
duke_all_tranfered_split = transfered_split(duke_all_dataset_path)

In [16]:
def pick_and_display_images(image_lists, num_lists=6, save_dir='all_cam_visualize/sample.jpg'):
    
    # Pick 6 random lists
    selected_lists = random.sample(image_lists, num_lists)
    
    # Open and resize images, store in a list of lists
    resized_images = []
    for img_list in selected_lists:
        row_images = [Image.open(img_path).resize((128, 256)) for img_path in img_list]
        resized_images.append(row_images)
    
    # Calculate the size of the final merged image
    total_width = 128 * len(selected_lists[0])
    total_height = 256 * num_lists
    
    # Create a new blank image with the calculated size
    merged_image = Image.new('RGB', (total_width, total_height))
    
    # Paste the images into the merged image
    for row_index, row_images in enumerate(resized_images):
        for col_index, img in enumerate(row_images):
            merged_image.paste(img, (col_index * 128, row_index * 256))
    merged_image.save(save_dir) 
    # Display the merged image
    #merged_image.show()

In [18]:
pick_and_display_images(list(market_all_tranfered_split.values()),3, 'all_cam_visualize/all_cam_market.jpg')


In [9]:
pick_and_display_images(list(duke_all_tranfered_split.values()),3, 'all_cam_visualize/all_cam_duke.jpg')

In [157]:
market_dataset_path = Path('/Volumes/NVMESSD1TB/PersonReID_Research/ReID_Dataset/Market-1501-v15.09.15')
market_person_cam_split = person_cam_split(market_dataset_path, 6)
market_full_cam = cam_filter(market_person_cam_split, 6)
market_cam_list = cam_split(market_dataset_path, 6)
cam_split_dataset(market_full_cam, './cam_split_market')

In [203]:
duke_dataset_path = Path('/Volumes/NVMESSD1TB/PersonReID_Research/ReID_Dataset/DukeMTMC-reID')
duke_person_cam_split = person_cam_split(duke_dataset_path, 8)
duke_full_cam = cam_filter(duke_person_cam_split, 4)
duke_cam_list = cam_split(duke_dataset_path, 8)
cam_split_dataset(duke_full_cam, './cam_split_duke2')

In [35]:
def display_random_images(image_lists):
    # Create a figure to display multiple images
    fig, axes = plt.subplots(1, len(image_lists), figsize=(15, 5))

    for i, image_list in enumerate(image_lists):
        # Pick a random image from the current sublist
        random_image_path = random.choice(image_list)
        
        # Load and display the image
        img = mpimg.imread(random_image_path)
        axes[i].imshow(img)
        axes[i].axis('off')  # Hide the axis

    # Adjust layout and show the images
    plt.tight_layout()
    plt.show()


In [40]:
def display_random_images_from_dict(person_dict, num_persons=6, num_lists=6):
    # Randomly select num_persons keys (person_ids) from the dictionary
    selected_persons = random.sample(list(person_dict.keys()), num_persons)
    
    for person_id in selected_persons:
        image_lists = person_dict[person_id]
        
        # Ensure there are enough lists to sample
        if len(image_lists) < num_lists:
            print(f"Not enough image lists for person {person_id}")
            continue
        
        # Randomly select num_lists sublists
        selected_lists = random.sample(image_lists, num_lists)
        
        # Create a figure to display multiple images
        fig, axes = plt.subplots(1, num_lists, figsize=(15, 5))
        
        for i, image_list in enumerate(selected_lists):
            # Pick a random image from the current sublist
            random_image_path = random.choice(image_list)
            
            # Load and display the image
            img = mpimg.imread(random_image_path)
            axes[i].imshow(img)
            axes[i].axis('off')  # Hide the axis
        
        # Adjust layout and show the images
        plt.tight_layout()
        plt.suptitle(f'Person ID: {person_id}')
        plt.show()

In [144]:
def cam_visualize(cam_list, images_per_cam=6):
    selected_images = []
    
    for camera_images in cam_list:
        # Ensure there are enough images in each camera view
        if len(camera_images) < images_per_cam:
            print(f"Not enough images in camera view: {camera_images}")
            continue
        
        # Randomly select num_images_per_camera images from each camera view
        selected_images.append(random.sample(camera_images, images_per_cam))
    
    # Determine the dimensions for the big image
    img_width, img_height = 128, 256
    num_cameras = len(selected_images)
    big_image_width = img_width * images_per_cam
    big_image_height = img_height * num_cameras
    
    # Create a new big image
    big_image = Image.new('RGB', (big_image_width, big_image_height))
    
    # Paste the selected images into the big image
    for i, camera_images in enumerate(selected_images):
        for j, image_path in enumerate(camera_images):
            # Load and resize the image
            img = Image.open(image_path)
            img = img.resize((img_width, img_height))
            
            # Calculate the position to paste the image
            x_offset = j * img_width
            y_offset = i * img_height
            
            # Paste the image into the big image
            big_image.paste(img, (x_offset, y_offset))
    
    # Display the big image
    plt.figure(figsize=(20, 20))
    plt.imshow(big_image)
    plt.axis('off')  # Hide the axis
    plt.show()

def cam_visualize_vertically(image_lists, num_images_per_camera=6):
    selected_images = []
    
    for camera_images in image_lists:
        # Ensure there are enough images in each camera view
        if len(camera_images) < num_images_per_camera:
            print(f"Not enough images in camera view: {camera_images}")
            continue
        
        # Randomly select num_images_per_camera images from each camera view
        selected_images.append(random.sample(camera_images, num_images_per_camera))
    
    # Determine the dimensions for the big image
    img_width, img_height = 128, 256
    num_cameras = len(selected_images)
    big_image_width = img_width * num_cameras
    big_image_height = img_height * num_images_per_camera
    
    # Create a new big image
    big_image = Image.new('RGB', (big_image_width, big_image_height))
    
    # Paste the selected images into the big image
    for i, camera_images in enumerate(selected_images):
        for j, image_path in enumerate(camera_images):
            # Load and resize the image
            img = Image.open(image_path)
            img = img.resize((img_width, img_height))
            
            # Calculate the position to paste the image
            x_offset = i * img_width
            y_offset = j * img_height
            
            # Paste the image into the big image
            big_image.paste(img, (x_offset, y_offset))
    
    # Display the big image
    plt.figure(figsize=(20, 20))
    plt.imshow(big_image)
    plt.axis('off')  # Hide the axis
    plt.show()


def display_random_images_from_dict(person_dict, num_persons=6, num_lists=6):
    # Randomly select num_persons keys (person_ids) from the dictionary
    selected_persons = random.sample(list(person_dict.keys()), num_persons)
    print(selected_persons)
    images = []
    
    for person_id in selected_persons:
        image_lists = list(person_dict[person_id].values())

        # Ensure there are enough lists to sample
        if len(image_lists) < num_lists:
            print(f"Not enough image lists for person {person_id}")
            continue
        
        # Randomly select num_lists sublists
        selected_lists = random.sample(image_lists, num_lists)
        
        for image_list in selected_lists:
            # Pick a random image from the current sublist
            random_image_path = random.choice(image_list)
            # Load the image
            img = Image.open(random_image_path)
            # Resize the image to 256x128
            img = img.resize((128, 256))
            # Append the image to the list
            images.append(img)
    
    # Merge images into one big image
    total_width = 128 * num_lists
    total_height = 256 * num_persons
    big_image = Image.new('RGB', (total_width, total_height))
    
    y_offset = 0
    for i in range(num_persons):
        x_offset = 0
        for j in range(num_lists):
            big_image.paste(images[i * num_lists + j], (x_offset, y_offset))
            x_offset += 128
        y_offset += 256
    
    # Display the big image
    plt.figure(figsize=(15, 15))
    plt.imshow(big_image)
    plt.axis('off')
    plt.show()